## Scikit-Learn은 특성변수 x가 2차원 자료만 작동한다

In [2]:
# p21
# 범주형 자료의 수량화
x = [{'city':'Seoul', 'temp':10.0}, {'city':'Dubai', 'temp':33.5}, {'city':'LA', 'temp':20.0}]
x

[{'city': 'Seoul', 'temp': 10.0},
 {'city': 'Dubai', 'temp': 33.5},
 {'city': 'LA', 'temp': 20.0}]

In [5]:
# city라는 key로 분류한 Seoul, Dubai, LA를 각각 범주형 수량화 자료로 변환
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
vec.fit_transform(x)
# x를 범주형 수량화 자료로 변환 

# 이와 같은 수량화를 one-hot 벡터라고 함.
# 범주형 자료로의 수량화 순서는 알파벳 순서
# sparse=False를 부여함으로써 이와 같은 범주형 수량화 결과를 볼 수 있음.

array([[ 0. ,  0. ,  1. , 10. ],
       [ 1. ,  0. ,  0. , 33.5],
       [ 0. ,  1. ,  0. , 20. ]])

In [6]:
# 메모리를 줄이기 위해 sparse=True
vec1 = DictVectorizer()
x1 = vec1.fit_transform(x)
x1

# DictVectorizer 클래스는 sparse=True가 디폴트
# 객체화 vec1은 sparse 행렬이며 오직 6개만 0이 아니므로 6개의 위치 key와 대응되는 값이 저장되었다는 메시지를 출력

<3x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [7]:
# sparse 압축을 풀어서 원래의 범주형 자료를 출력하고 특성변수의 이름을 확인하고 싶을 때

# 배열로 변경
x1.toarray()
vec1.get_feature_names()

['city=Dubai', 'city=LA', 'city=Seoul', 'temp']

In [8]:
# 텍스트 자료의 수량화
text = ['떴다 떴다 비행기 날아라 날아라',
        '높이 높이 날아라 우리 비행기',
        '내가 만든 비행기 날아라 날아라',
        '멀리 멀리 날아라 우리 비행기']
    
text

['떴다 떴다 비행기 날아라 날아라',
 '높이 높이 날아라 우리 비행기',
 '내가 만든 비행기 날아라 날아라',
 '멀리 멀리 날아라 우리 비행기']

In [9]:
# text의 수량화는 CountVectorizer 클래스를 이용함
from sklearn.feature_extraction.text import CountVectorizer
vec2 = CountVectorizer()
# CountVectorizer는 디폴트가 sparse=True이며 sparse 압축을 풀기 위해서는 toarray 함수가 필요함.

# text를 수량화 배열 자료로 변환
t = vec2.fit_transform(text).toarray()

import pandas as pd
t1 = pd.DataFrame(t, columns=vec2.get_feature_names())
t1

# 문장이 4문장이므로 0 1 2 3 이렇게 뜸
# ex) 1번째 문장에서는 날아라 2개, 떴다 2개, 비행기 1개가 뜸.

,날아라,내가,높이,떴다,만든,멀리,비행기,우리
0,2,0,0,2,0,0,1,0
1,1,0,2,0,0,0,1,1
2,2,1,0,0,1,0,1,0
3,1,0,0,0,0,2,1,1


In [10]:
# TF-IDF (Term Frequency Inverse Document Frequency)

# 전체 문서에서 출현 비중이 높은 단어에는 상대적으로 낮은 가중치(Weight)를 주고
# 출현 비중이 낮은 단어에는 상대적으로 높은 가중치를 부여

from sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer()
# 높은 빈도는 낮은 가중치, 낮은 빈도는 높은 가중치
x2 = tfid.fit_transform(text).toarray()
x3 = pd.DataFrame(x2, columns=tfid.get_feature_names())
x3

,날아라,내가,높이,떴다,만든,멀리,비행기,우리
0,0.450735,0.000000,0.00000,0.86374,0.000000,0.00000,0.225368,0.000000
1,0.229589,0.000000,0.87992,0.00000,0.000000,0.00000,0.229589,0.346869
2,0.569241,0.545415,0.00000,0.00000,0.545415,0.00000,0.284620,0.000000
3,0.229589,0.000000,0.00000,0.00000,0.000000,0.87992,0.229589,0.346869


In [12]:
# 특성변수 생성

import plotly.express as px
import numpy as np

X = np.array([1, 2, 3, 4, 5])
Y = np.array([5, 3, 1, 5, 8])

# 선형회귀를 하기에는 부적합
fig = px.line(x=X, y=Y)
fig.update_layout(width=500, height=380)
fig.show()

In [19]:
# 특성변수의 생성
# x^2와 x^3을 특성변수로 추가하여 이를 선형모형으로 적합하는 방법을 고려해보자.

from sklearn.preprocessing import PolynomialFeatures

# degree=3을 부여하여 x^2, x^3을 새로운 특성변수로 생성
fg = PolynomialFeatures(degree=3, include_bias=True)
# include_bias는 절편항 여부
# 첫 열은 모두 1로 y절편(intercept term)을 추가한 것
# Y절편을 포함하고 싶지 않은 경우, fg=PolynomialFeatures(degree=3, include_bias=False)로 수정

# 3차까지 생성
x1 = fg.fit_transform(X[:, np.newaxis])
x1

array([[  1.,   1.,   1.,   1.],
       [  1.,   2.,   4.,   8.],
       [  1.,   3.,   9.,  27.],
       [  1.,   4.,  16.,  64.],
       [  1.,   5.,  25., 125.]])

In [21]:
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go

reg = LinearRegression()
reg.fit(x1, Y)

# 적합값
yfit = reg.predict(x1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=Y, mode='lines', name='origin'))
fig.add_trace(go.Scatter(x=X, y=yfit, mode='lines+markers', name='fitted'))
fig.show()

In [23]:
# 결측자료 대체
# Numpy에서는 np.nan, np.NaN, None이 모두 결측치를 의미

x_miss = np.array([[1, 2, 3, None], [5, np.NaN, 7, 8], [None, 10, 11, 12], [13, np.nan, 15, 16]])
x_miss

array([[1, 2, 3, None],
       [5, nan, 7, 8],
       [None, 10, 11, 12],
       [13, nan, 15, 16]], dtype=object)

In [25]:
from sklearn.impute import SimpleImputer
im = SimpleImputer(strategy='mean')
# 열의 평균값으로 대체
im.fit_transform(x_miss)

# 각 특성변수 x1, x2, x3, x4 별로 각각의 평균값으로 결측치를 대체

array([[ 1.        ,  2.        ,  3.        , 12.        ],
       [ 5.        ,  6.        ,  7.        ,  8.        ],
       [ 6.33333333, 10.        , 11.        , 12.        ],
       [13.        ,  6.        , 15.        , 16.        ]])

In [29]:
from sklearn.impute import SimpleImputer
im2 = SimpleImputer(strategy='median')
# 열의 median으로 대체
im2.fit_transform(x_miss)

array([[ 1.,  2.,  3., 12.],
       [ 5.,  6.,  7.,  8.],
       [ 5., 10., 11., 12.],
       [13.,  6., 15., 16.]])

In [30]:
# Pipeline Library를 이용한 결측 자료 대체 및 특성변수 생성

import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

y = pd.Series([2, 5, 1, 6])
model = make_pipeline(SimpleImputer(strategy='mean'), PolynomialFeatures(degree=2), LinearRegression())
model.fit(x_miss, y)
# 이 자료에서는 관측치는 4개, 특성변수 14개로 완전 적합됨
model.predict(x_miss)

array([2., 5., 1., 6.])

## Scikit-Learn을 적용하여 본격적인 자료분석을 하기 전에 특성변수의 선택, 특성변수의 특징, 이상치의 존재유무, 특성변수들 간의 관계 등을 파악해야 한다.